# Reference

In [1]:
from piper.defaults import *
from piper.styler import *

piper v0.1.0: Monday, 29 March 2021 19:08:19


# Data example

In [2]:
# https://youtu.be/_5EhKDCm9VY

In [3]:
headers = ['account', 'type', 'balance']
headers2 = [x + '2' for x in headers]

f = 'inputs/data merge testing.xlsx'
sheet_name = 'Example #1'

df1 = pd.read_excel(f, sheet_name=sheet_name, usecols='A:C')
display(df1.head())
df2 = pd.read_excel(f, sheet_name=sheet_name, usecols='F:H', header=0, names=headers2)
display(df2.head())

,account,type,balance
0,1,checking,87
1,2,checking,36
2,3,checking,82
3,4,checking,76
4,5,savings,76


,account2,type2,balance2
0,1,checking,87
1,2,savings,36
2,3,checking,82
3,4,checking,76
4,8,savings,76


In [4]:
merge_df = (pd.merge(df1, df2, how='outer',
                    left_on=['account'], right_on=['account2']))
merge_df

,account,type,balance,account2,type2,balance2
0,1,checking,87,1.0,checking,87.0
1,2,checking,36,2.0,savings,36.0
2,3,checking,82,3.0,checking,82.0
3,4,checking,76,4.0,checking,76.0
4,5,savings,76,5.0,checking,33.0
5,6,checking,27,6.0,checking,43.0
6,7,checking,42,7.0,checking,27.0
7,8,checking,33,8.0,savings,76.0
8,9,checking,18,9.0,checking,18.0
9,10,checking,54,NaN,NaN,NaN


# Styling examples

### highlight values - string values

In [5]:
sample_data = sample(merge_df, 10)

(sample_data.style.apply(highlight_values, values='savings', subset=['type', 'type2'])
                  .apply(highlight_values, values=['checking'], subset=['type', 'type2'],
                         css='color: blue')
                  .apply(highlight_values, values=27, subset=['balance2'])
                  .hide_index()
)

14 rows, 6 columns


account,type,balance,account2,type2,balance2
9,checking,18,9.000000,checking,18.000000
2,checking,36,2.000000,savings,36.000000
1,checking,87,1.000000,checking,87.000000
6,checking,27,6.000000,checking,43.000000
13,savings,20,13.000000,savings,20.000000
12,savings,31,12.000000,savings,33.000000
13,savings,20,13.000000,checking,54.000000
7,checking,42,7.000000,checking,27.000000
11,savings,16,11.000000,savings,30.000000
5,savings,76,5.000000,checking,33.000000


### highlight values - numeric values

In [6]:
sample_data = sample(merge_df, 10)

(sample_data.style.apply(highlight_values, values=[20, 54], subset=['balance', 'balance2'],
                         css='color: purple')
                  .apply(highlight_values, values=[20, 54], subset=['balance', 'balance2'], 
                         css='background: orange')
                  .apply(highlight_values, values=[33, 76], subset=['balance'],
                                  css='color: yellow; background:blue')
                  .apply(highlight_values, values=[4, 13], subset=['account'],
                                  css='color: red; background: lightpink')
                  .hide_index()
)

14 rows, 6 columns


account,type,balance,account2,type2,balance2
12,savings,31,12.000000,savings,33.000000
5,savings,76,5.000000,checking,33.000000
13,savings,20,13.000000,checking,54.000000
11,savings,16,11.000000,savings,30.000000
3,checking,82,3.000000,checking,82.000000
4,checking,76,4.000000,checking,76.000000
8,checking,33,8.000000,savings,76.000000
2,checking,36,2.000000,savings,36.000000
6,checking,27,6.000000,checking,43.000000
1,checking,87,1.000000,checking,87.000000


#### highlight values - gt, lt, ge, le

In [7]:
sample_data = sample(merge_df, 10)

(sample_data.style.apply(highlight_values, values=20,
                         subset=['balance', 'balance2'],
                         css='color: purple')
                  .hide_index()
)

14 rows, 6 columns


account,type,balance,account2,type2,balance2
6,checking,27,6.000000,checking,43.000000
2,checking,36,2.000000,savings,36.000000
7,checking,42,7.000000,checking,27.000000
1,checking,87,1.000000,checking,87.000000
12,savings,31,12.000000,savings,33.000000
5,savings,76,5.000000,checking,33.000000
13,savings,20,13.000000,checking,54.000000
9,checking,18,9.000000,checking,18.000000
8,checking,33,8.000000,savings,76.000000
3,checking,82,3.000000,checking,82.000000


### highlight null values

In [8]:
sample_data = sample(merge_df, 10)

(sample_data.style.hide_index()
                  .highlight_null(null_color='lightgreen')
)

14 rows, 6 columns


account,type,balance,account2,type2,balance2
2,checking,36,2.000000,savings,36.000000
6,checking,27,6.000000,checking,43.000000
8,checking,33,8.000000,savings,76.000000
7,checking,42,7.000000,checking,27.000000
11,savings,16,11.000000,savings,30.000000
13,savings,20,13.000000,checking,54.000000
5,savings,76,5.000000,checking,33.000000
9,checking,18,9.000000,checking,18.000000
12,savings,31,12.000000,savings,33.000000
13,savings,20,13.000000,savings,20.000000


### Highlight (null row) values

In [9]:
sample_data = sample(merge_df, 10)

(sample_data.style
            .hide_index()
            .apply(highlight_rows, 
                   column='account2', 
                   operator='null',
                   css='color: blue; background: lightgrey',
                   axis=1)  
)

14 rows, 6 columns


account,type,balance,account2,type2,balance2
3,checking,82,3.000000,checking,82.000000
8,checking,33,8.000000,savings,76.000000
5,savings,76,5.000000,checking,33.000000
13,savings,20,13.000000,savings,20.000000
2,checking,36,2.000000,savings,36.000000
1,checking,87,1.000000,checking,87.000000
9,checking,18,9.000000,checking,18.000000
6,checking,27,6.000000,checking,43.000000
4,checking,76,4.000000,checking,76.000000
7,checking,42,7.000000,checking,27.000000


### Highlight (True/False) values

In [10]:
merge_df['account2 is null'] = merge_df['account2'].isna()

sample_data = sample(merge_df, 10)

(sample_data.style.apply(highlight_values, values=False,
                         subset=['account2 is null'],
                         css='color: yellow; background: blue')
 .apply(highlight_values, values=True,
                         subset=['account2 is null'],
                         css='color: yellow; background: red')
)

14 rows, 7 columns


,account,type,balance,account2,type2,balance2,account2 is null
5,6,checking,27,6.000000,checking,43.000000,False
9,10,checking,54,nan,nan,nan,True
0,1,checking,87,1.000000,checking,87.000000,False
4,5,savings,76,5.000000,checking,33.000000,False
7,8,checking,33,8.000000,savings,76.000000,False
2,3,checking,82,3.000000,checking,82.000000,False
8,9,checking,18,9.000000,checking,18.000000,False
1,2,checking,36,2.000000,savings,36.000000,False
6,7,checking,42,7.000000,checking,27.000000,False
12,13,savings,20,13.000000,checking,54.000000,False


### Background gradient values (subset column)

In [11]:
(merge_df.head().pipe(pd.DataFrame.sort_values, by='balance2', ascending=False)
                           .style                     
                           .background_gradient(subset='balance2', cmap='coolwarm')
                           .hide_index()
)

account,type,balance,account2,type2,balance2,account2 is null
1,checking,87,1.000000,checking,87.000000,False
3,checking,82,3.000000,checking,82.000000,False
4,checking,76,4.000000,checking,76.000000,False
2,checking,36,2.000000,savings,36.000000,False
5,savings,76,5.000000,checking,33.000000,False


### Background gradient (heatmap)

In [12]:
merge_df.style.background_gradient(cmap='viridis').hide_index()

account,type,balance,account2,type2,balance2,account2 is null
1,checking,87,1.000000,checking,87.000000,False
2,checking,36,2.000000,savings,36.000000,False
3,checking,82,3.000000,checking,82.000000,False
4,checking,76,4.000000,checking,76.000000,False
5,savings,76,5.000000,checking,33.000000,False
6,checking,27,6.000000,checking,43.000000,False
7,checking,42,7.000000,checking,27.000000,False
8,checking,33,8.000000,savings,76.000000,False
9,checking,18,9.000000,checking,18.000000,False
10,checking,54,nan,nan,nan,True


### Highlight max/min values

In [13]:
(merge_df.sort_values(by='balance2', ascending=True).style
    .apply(highlight_max, subset='balance', css='background: orange') 
    .apply(highlight_max, subset='balance2', css='background: orange') 
    .apply(highlight_min, subset='balance', css='background: lightblue')  
    .apply(highlight_min, subset='balance2', css='background: lightblue') 
    .hide_index()
)

account,type,balance,account2,type2,balance2,account2 is null
9,checking,18,9.000000,checking,18.000000,False
13,savings,20,13.000000,savings,20.000000,False
7,checking,42,7.000000,checking,27.000000,False
11,savings,16,11.000000,savings,30.000000,False
5,savings,76,5.000000,checking,33.000000,False
12,savings,31,12.000000,savings,33.000000,False
2,checking,36,2.000000,savings,36.000000,False
6,checking,27,6.000000,checking,43.000000,False
13,savings,20,13.000000,checking,54.000000,False
4,checking,76,4.000000,checking,76.000000,False


## Highlight max/min values - standalone example

In [14]:
import pandas as pd
import numpy as np

np.random.seed(24)
df = pd.DataFrame({'A': np.linspace(1, 10, 10)})
df2 = pd.DataFrame(np.random.randn(10, 4), columns=list('BCDE'))

df = pd.concat([df, df2], axis=1)
df.iloc[0, 2] = np.nan

(df.style.apply(highlight_min, subset=['B', 'E'], axis=0)
         .apply(highlight_min, subset=['A', 'C'], axis=0))

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,-0.481165,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


### Column value formatting options

In [15]:
(merge_df[:8].style
              .format({"account": "{:.0f}", "balance": "{:.0f}",
                       "account2": "{:.6f}", "balance2": "{:.0f}"})
              .hide_index()
              .set_caption("This is a caption that is modified")
              .set_table_styles([{'selector': 'caption',
                                  'props': [('color', 'blue'),
                                            ('font-size', '18px')]}])
)

account,type,balance,account2,type2,balance2,account2 is null
1,checking,87,1.000000,checking,87,False
2,checking,36,2.000000,savings,36,False
3,checking,82,3.000000,checking,82,False
4,checking,76,4.000000,checking,76,False
5,savings,76,5.000000,checking,33,False
6,checking,27,6.000000,checking,43,False
7,checking,42,7.000000,checking,27,False
8,checking,33,8.000000,savings,76,False


### data bars 

In [16]:
(merge_df.sort_values('balance2', ascending=False)
 .style
 .bar(subset=["account",], color='lightgreen')
 .bar(subset=["balance2",], color='lightblue')
 .format({"account": "{:.0f}", "balance": "{:.0f}",
          "account2": "{:.4f}", "balance2": "{:.0f}"})
 .hide_index()
 .highlight_null(null_color='blue')
 .highlight_max(subset='balance', color='lightblue')
 .highlight_min(color='#cd4f39')
 .set_caption('this is a test')
)

account,type,balance,account2,type2,balance2,account2 is null
1,checking,87,1.0000,checking,87,False
3,checking,82,3.0000,checking,82,False
4,checking,76,4.0000,checking,76,False
8,checking,33,8.0000,savings,76,False
13,savings,20,13.0000,checking,54,False
6,checking,27,6.0000,checking,43,False
2,checking,36,2.0000,savings,36,False
5,savings,76,5.0000,checking,33,False
12,savings,31,12.0000,savings,33,False
11,savings,16,11.0000,savings,30,False


### highlight rows (including null values)

In [17]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'A': np.linspace(1, 10, 10)})
random_int = np.random.randint(1, 10)
random_matrix = pd.DataFrame(np.random.randn(10, 4) + random_int, columns=list('BCDE'))
df = pd.concat([df, random_matrix], axis=1)
df.iloc[3, 2] = np.nan

(df.style.hide_index()
         .apply(highlight_rows, 
                column='B', operator='>', value=9,
                css='background: lightblue',
                axis=1)  
)

A,B,C,D,E
1.000000,7.608854,8.282859,9.583753,5.480171
2.000000,5.708928,4.596789,6.040570,7.045512
3.000000,8.192646,9.031613,5.286531,6.527766
4.000000,7.549451,nan,7.094302,7.740124
5.000000,6.500995,6.783348,7.470376,6.810669
6.000000,6.375782,6.128886,6.020285,7.509046
7.000000,7.271534,8.154401,5.837164,7.291061
8.000000,7.351390,6.899540,7.929316,7.935968
9.000000,5.919889,7.021434,6.815929,6.230159
10.000000,5.471785,5.489694,6.602862,6.703364


In [18]:
(df.style.hide_index()
         .highlight_null(null_color='yellow')
         .apply(highlight_rows, column='C', operator='null', axis=1)   
)

A,B,C,D,E
1.000000,7.608854,8.282859,9.583753,5.480171
2.000000,5.708928,4.596789,6.040570,7.045512
3.000000,8.192646,9.031613,5.286531,6.527766
4.000000,7.549451,nan,7.094302,7.740124
5.000000,6.500995,6.783348,7.470376,6.810669
6.000000,6.375782,6.128886,6.020285,7.509046
7.000000,7.271534,8.154401,5.837164,7.291061
8.000000,7.351390,6.899540,7.929316,7.935968
9.000000,5.919889,7.021434,6.815929,6.230159
10.000000,5.471785,5.489694,6.602862,6.703364


## DataFrame formats

### AS/400 Style

In [19]:
merge_df.style.set_table_styles(get_style())

,account,type,balance,account2,type2,balance2,account2 is null
0,1,checking,87,1.000000,checking,87.000000,False
1,2,checking,36,2.000000,savings,36.000000,False
2,3,checking,82,3.000000,checking,82.000000,False
3,4,checking,76,4.000000,checking,76.000000,False
4,5,savings,76,5.000000,checking,33.000000,False
5,6,checking,27,6.000000,checking,43.000000,False
6,7,checking,42,7.000000,checking,27.000000,False
7,8,checking,33,8.000000,savings,76.000000,False
8,9,checking,18,9.000000,checking,18.000000,False
9,10,checking,54,nan,nan,nan,True


In [20]:
xl_output = (head(merge_df, 8).style.set_properties(**get_style(style='ibm'))
                  .hide_index()
                  .apply(highlight_max,
                         subset=['account2', 'balance', 'account2 is null'],
                         css='color: yellow'))
xl_output

14 rows, 7 columns


account,type,balance,account2,type2,balance2,account2 is null
1,checking,87,1.000000,checking,87.000000,False
2,checking,36,2.000000,savings,36.000000,False
3,checking,82,3.000000,checking,82.000000,False
4,checking,76,4.000000,checking,76.000000,False
5,savings,76,5.000000,checking,33.000000,False
6,checking,27,6.000000,checking,43.000000,False
7,checking,42,7.000000,checking,27.000000,False
8,checking,33,8.000000,savings,76.000000,False


### Alternative style

In [21]:
data = head(merge_df, 8)
     
xl_output = (data.style.set_table_styles(get_style())
             .apply(highlight_min, subset='balance', css='color: yellow')
             .apply(highlight_min, subset='balance', css='background: blue')
             .apply(highlight_max, subset='balance', css='color: yellow')
             .apply(highlight_max, subset='balance', css='color: blue; background: lightgreen')
             .set_caption("This is a caption"))
display(xl_output)

xl_output = data.style.set_table_styles(get_style()).hide_index()
display(xl_output)

xx = df.groupby(['A', 'B','C']).agg(Total=pd.NamedAgg('D', 'sum'))
xx.style.set_table_styles(get_style())

14 rows, 7 columns


,account,type,balance,account2,type2,balance2,account2 is null
0,1,checking,87,1.000000,checking,87.000000,False
1,2,checking,36,2.000000,savings,36.000000,False
2,3,checking,82,3.000000,checking,82.000000,False
3,4,checking,76,4.000000,checking,76.000000,False
4,5,savings,76,5.000000,checking,33.000000,False
5,6,checking,27,6.000000,checking,43.000000,False
6,7,checking,42,7.000000,checking,27.000000,False
7,8,checking,33,8.000000,savings,76.000000,False


account,type,balance,account2,type2,balance2,account2 is null
1,checking,87,1.000000,checking,87.000000,False
2,checking,36,2.000000,savings,36.000000,False
3,checking,82,3.000000,checking,82.000000,False
4,checking,76,4.000000,checking,76.000000,False
5,savings,76,5.000000,checking,33.000000,False
6,checking,27,6.000000,checking,43.000000,False
7,checking,42,7.000000,checking,27.000000,False
8,checking,33,8.000000,savings,76.000000,False


,,,Total
A,B,C,
1.0,7.608853704525917,8.282859086825983,9.583753
2.0,5.7089277765494675,4.59678887662051,6.040570
3.0,8.192646442077343,9.031613480164705,5.286531
5.0,6.500995357544943,6.78334839355736,7.470376
6.0,6.375781815355552,6.128886032967243,6.020285
7.0,7.27153411301993,8.154401089388678,5.837164
8.0,7.35138956356561,6.89954018960505,7.929316
9.0,5.919889219662041,7.021434048383398,6.815929
10.0,5.471785321131271,5.4896935684299395,6.602862


In [22]:
# xl_output.to_excel('outputs/test.xlsx', index=False, engine='openpyxl')

In [23]:
head(df)

10 rows, 5 columns


,A,B,C,D,E
0,1.0,7.608854,8.282859,9.583753,5.480171
1,2.0,5.708928,4.596789,6.040570,7.045512
2,3.0,8.192646,9.031613,5.286531,6.527766
3,4.0,7.549451,NaN,7.094302,7.740124
